In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)
from datetime import datetime, date, time, timezone
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

import mglearn
from IPython.display import display

import glob
import os

In [87]:
df = pd.read_csv('/Users/elle.le/Workspace/BoxSize_local/new_data/all_item.csv')
df.drop(columns=['Unnamed: 0'], inplace = True)
#df['weight'] = df['weight'] * 1000
df['name_split'] = df['product_name'].str.split(' ')
df.dropna(subset=['name_split','weight'], inplace = True)
df.reset_index(inplace=True,drop=True)
print(df.shape)
df.head(5)

(14748, 13)


,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm,item_vol,item_label,product_name,variant_name,name_string,brand_name,brand_id,weight,name_split
0,126,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Taupe,anastasia-beverly-hills-perfect-brow-pencil-taupe,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
1,127,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Blonde,anastasia-beverly-hills-perfect-brow-pencil-bl...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
2,128,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Caramel,anastasia-beverly-hills-perfect-brow-pencil-ca...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
3,129,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Medium Brown,anastasia-beverly-hills-perfect-brow-pencil-me...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"
4,130,18.0,18.0,172.0,3.34368,3.0,Perfect Brow Pencil,Dark Brown,anastasia-beverly-hills-perfect-brow-pencil-da...,Anastasia Beverly Hills,117.0,0.03,"[Perfect, Brow, Pencil]"


In [118]:
df_train_no_na = df.copy()
df_train_no_na.dropna(subset=['dim_packaging_width_mm'], inplace = True)
df_train_no_na.reset_index(drop=True, inplace = True)
df_train_no_na.shape

(10475, 13)

In [119]:
df_fit_na = df.loc[df['dim_packaging_width_mm'].isna()].copy()
df_fit_na.shape


(4273, 13)

In [120]:
# Group products using similarities in their name and weight
# Create a list of all words used in product name

total_word_list =[]
for i in range(df.shape[0]):
    total_word_list += df.iloc[i,-1] #Make sure the last columns is the name_split column
    
unique_word = np.unique(total_word_list)    
len(unique_word)

3693

In [121]:
# Group products using similarities in their name and weight
# Create a list of all words used in product name

total_word_list =[]
for i in range(df.shape[0]):
    total_word_list += df.iloc[i,-1] #Make sure the last columns is the name_split column
    
unique_word = np.unique(total_word_list)    
len(unique_word)

3693

In [122]:
#Name matrix for NOT NA only

name_matrix = np.zeros((df_train_no_na.shape[0], len(unique_word)))
name_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [123]:
word_col_map = dict(zip(unique_word,range(len(unique_word))))

In [124]:
for i in range(df_train_no_na.shape[0]):
    words = df.iloc[i,-1]
    for word in words:
        name_matrix[i,word_col_map[word]] = 1
        
name_matrix 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [125]:
type(name_matrix)

numpy.ndarray

In [126]:
feature_df=pd.DataFrame(name_matrix)
feature_df

,0,1,2,3,4,5,6,7,8,9,...,3683,3684,3685,3686,3687,3688,3689,3690,3691,3692
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
feature_df[['variant_id','weight']] = df_train_no_na[['variant_id','weight']]
feature_df['weight'] = feature_df['weight']*1000
feature_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,3685,3686,3687,3688,3689,3690,3691,3692,variant_id,weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126,30.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127,30.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128,30.0


In [135]:
name_matrix_na = np.zeros((df_fit_na.shape[0], len(unique_word)))

for i in range(df_fit_na.shape[0]):
    words = df.iloc[i,-1]
    for word in words:
        name_matrix_na[i,word_col_map[word]] = 1
        
feature_df_na=pd.DataFrame(name_matrix_na)


feature_df_na[['variant_id','weight']] = df_fit_na[['variant_id','weight']]
feature_df_na['weight'] = feature_df_na['weight']*1000
feature_df_na.fillna(0, inplace = True)
print(feature_df_na.shape)
feature_df_na.head(3)



(4273, 3695)


,0,1,2,3,4,5,6,7,8,9,...,3685,3686,3687,3688,3689,3690,3691,3692,variant_id,weight
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
#KNN
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=15, algorithm='brute').fit(feature_df)

#df_fit_na
distances, indices = nbrs.kneighbors(feature_df_na)
indices

array([[   0,    1,    2, ...,   15,   12,   13],
       [   0,    1,    2, ...,   15,   12,   13],
       [   0,    1,    2, ...,   15,   12,   13],
       ...,
       [2995, 2994, 2993, ..., 2997, 2984, 2983],
       [2998, 2997, 2995, ..., 2996, 2985, 2984],
       [2998, 2997, 2995, ..., 2996, 2985, 2984]])

In [150]:
distances

array([[129.52219887, 130.49521064, 131.46862744, ..., 156.22099731,
        156.2305988 , 157.17824277],
       [129.52219887, 130.49521064, 131.46862744, ..., 156.22099731,
        156.2305988 , 157.17824277],
       [129.52219887, 130.49521064, 131.46862744, ..., 156.22099731,
        156.2305988 , 157.17824277],
       ...,
       [ 73.8241153 ,  74.66592262,  75.51820972, ...,  84.1546196 ,
         84.16650165,  85.0470458 ],
       [ 49.47726751,  50.06995107,  66.05300902, ...,  76.36753237,
         77.04544114,  78.05767099],
       [ 50.67543784,  51.29327441,  68.05145112, ...,  78.07688518,
         79.04429138,  80.05623024]])

In [141]:
len(np.unique(indices.reshape(-1)))

1707

In [152]:
print(indices[500,:], distances[500,:])

[167 166 173 174 165 175 164 163 162 161 160 159 158 157 176] [ 44.77722635  46.57252409  63.8435588   69.77105417  77.11031059
  77.6916984   78.95568377  88.57764955  90.36038955  93.96807969
  98.5139584  100.34440692 101.25709852 102.1763182  105.8820098 ]


In [161]:
np.where(feature_df.iloc[261,:] > 0)

(array([1307, 1449, 2918, 3203, 3693, 3694]),)

In [158]:
df_train_no_na.loc[df_train_no_na['product_name'].str.startswith('Nail Enamel'),:]

,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm,item_vol,item_label,product_name,variant_name,name_string,brand_name,brand_id,weight,name_split
261,29207,31.0,32.0,70.0,4.1664,0.0,Nail Enamel,703 Matte,inglot-cosmetics-nail-enamel-703-matte,Inglot Cosmetics,1115.0,0.13,"[Nail, Enamel]"
262,29245,31.0,32.0,70.0,4.1664,0.0,Nail Enamel,21,inglot-cosmetics-nail-enamel-021,Inglot Cosmetics,1115.0,0.13,"[Nail, Enamel]"
263,29271,31.0,32.0,70.0,4.1664,0.0,Nail Enamel,139,inglot-cosmetics-nail-enamel-139,Inglot Cosmetics,1115.0,0.13,"[Nail, Enamel]"


In [151]:
df_fit_na.iloc[500,:]

variant_id                                            29262
dim_packaging_length_mm                                 NaN
dim_packaging_width_mm                                  NaN
dim_packaging_height_mm                                 NaN
item_vol                                                NaN
item_label                                              NaN
product_name                                    Nail Enamel
variant_name                                            112
name_string                inglot-cosmetics-nail-enamel-112
brand_name                                 Inglot Cosmetics
brand_id                                               1115
weight                                                 0.13
name_split                                   [Nail, Enamel]
Name: 763, dtype: object

In [153]:
df_train_no_na.iloc[indices[500,:],:]

,variant_id,dim_packaging_length_mm,dim_packaging_width_mm,dim_packaging_height_mm,item_vol,item_label,product_name,variant_name,name_string,brand_name,brand_id,weight,name_split
167,27872,0.0,0.0,0.0,0.00000,0.0,Freedom System Blush,37,inglot-cosmetics-freedom-system-face-blush-37,Inglot Cosmetics,1115.0,0.04,"[Freedom, System, Blush]"
166,27870,0.0,0.0,0.0,0.00000,0.0,Freedom System Blush,35,inglot-cosmetics-freedom-system-face-blush-35,Inglot Cosmetics,1115.0,0.04,"[Freedom, System, Blush]"
173,27975,23.0,23.0,97.0,3.07878,0.0,Cream Concealer,23,inglot-cosmetics-cream-concealer-23,Inglot Cosmetics,1115.0,0.05,"[Cream, Concealer]"
174,27981,23.0,23.0,97.0,3.07878,0.0,Cream Concealer,29,inglot-cosmetics-cream-concealer-29,Inglot Cosmetics,1115.0,0.05,"[Cream, Concealer]"
165,27841,15.0,15.0,144.0,1.94400,3.0,Full Metal Eyebrow Pencil,515,inglot-cosmetics-full-metal-eyebrow-pencil-515,Inglot Cosmetics,1115.0,0.03,"[Full, Metal, Eyebrow, Pencil]"
175,27989,23.0,23.0,97.0,3.07878,0.0,Cream Concealer,37,inglot-cosmetics-cream-concealer-37,Inglot Cosmetics,1115.0,0.05,"[Cream, Concealer]"
164,27839,15.0,15.0,144.0,1.94400,3.0,Full Metal Eyebrow Pencil,513,inglot-cosmetics-full-metal-eyebrow-pencil-513,Inglot Cosmetics,1115.0,0.03,"[Full, Metal, Eyebrow, Pencil]"
163,27833,16.0,16.0,134.0,2.05824,3.0,Eyebrow Pencil,501,inglot-cosmetics-eyebrow-pencil-501,Inglot Cosmetics,1115.0,0.02,"[Eyebrow, Pencil]"
162,27831,9.0,49.0,50.0,1.32300,0.0,Freedom System Brow Powder,568,inglot-cosmetics-brow-powder-square-568,Inglot Cosmetics,1115.0,0.02,"[Freedom, System, Brow, Powder]"
161,27827,9.0,49.0,50.0,1.32300,0.0,Freedom System Brow Powder,564,inglot-cosmetics-brow-powder-square-564,Inglot Cosmetics,1115.0,0.02,"[Freedom, System, Brow, Powder]"


In [ ]:
df_unique_name.iloc[14747,1:12]

In [ ]:
df_unique_name.iloc[14724,1:12]

In [ ]:
df_unique_name.iloc[na_item_group[500,:]]

In [ ]:
name_matrix[0,685]

In [ ]:
word_col_map['Perfect']

In [ ]:
word_col_map['Brow']

In [ ]:
# List of indices of SKUs without dimensions:

na_dim_indices = df['dim_packaging_length_mm'].isna()
na_dim_indices

In [ ]:
na_item_group = indices[na_dim_indices,:]
na_item_group

In [ ]:
df_test = df.copy()
for row in range(na_item_group.shape[0]):
    feature_median = df.iloc[na_item_group[row,:],[df.columns.get_loc(c) 
                             for c in ['dim_packaging_length_mm',
                                       'dim_packaging_width_mm',
                                       'dim_packaging_height_mm',
                                       'weight']]].median(axis=0)
    if row == 500:
        print('feature_median',feature_median.copy().to_numpy())
        print('na_item_group',na_item_group[row,:])
        print('pre-filled df_test',df_test.iloc[na_item_group[row,:],:])
    if not any(feature_median.isna()):
        df_test.iloc[na_item_group[row,:],[df_test.columns.get_loc(c) 
                             for c in ['dim_packaging_length_mm',
                                       'dim_packaging_width_mm',
                                       'dim_packaging_height_mm',
                                       'weight']]] = feature_median.copy().to_numpy()

    


In [ ]:
#df_test.iloc[na_item_group[500,:],1:4] = np.array([1,2,3])
df_test.iloc[na_item_group[500,:],:]

In [ ]:
#df.iloc[na_item_group[500,:],:]

In [ ]:
df_test.shape

In [ ]:
df_test.isna().sum()

In [ ]:
#df_test.loc[df_test['dim_packaging_width_mm'].isna()]

In [ ]:
indices[14746,:]

In [ ]:
indices

In [ ]:
df_test.loc[df_test['product_name'].str.startswith('Magic Star')]


In [ ]:
df.iloc[indices[14746,:],:]

In [157]:
X_train = np.array([[0, 0], [1, 0], [2, 0], [0, 1], [1, 1], [2, 1], [0,2], [1,2], [2,2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='brute').fit(X_train)
X_test = [[0.1, 0.1], [1.8, 1.8]]
distances, indices = nbrs.kneighbors(X_test)
print(indices)
print(distances)

[[0 1]
 [8 5]]
[[0.14142136 0.90553851]
 [0.28284271 0.82462113]]
